# Get the CVSS score of the vulnerability(CVE-ID) listed in the file 

In [1]:
import pandas as pd
df = pd.read_csv("95012IncludingCVEs_20230326.csv")
df.head()

,created_at,id,text,author_id,edit_history_tweet_ids,public_metrics,CVE
0,2023-02-06T15:00:04.000Z,1622611085894328322,Docker Full Privilege Escalation (#CVE-2022-25...,4.365682e+08,['1622611085894328322'],"{'retweet_count': 0, 'reply_count': 0, 'like_c...",CVE-2022-25365
1,2023-02-06T15:00:35.000Z,1622611217725399042,RT @0xor0ne: CVE-2021-27365: analysis and expl...,1.949128e+07,['1622611217725399042'],"{'retweet_count': 107, 'reply_count': 0, 'like...",CVE-2021-27365
2,2023-02-06T15:01:32.000Z,1622611455823732736,CVE-2021-21974 https://t.co/OqV4Ki8gRI #Harsia...,1.083501e+18,['1622611455823732736'],"{'retweet_count': 0, 'reply_count': 0, 'like_c...",CVE-2021-21974
3,2023-02-06T15:02:17.000Z,1622611643040423939,RT @SecurityBDog: Is CVE-2022-42889 important ...,1.359569e+18,['1622611643040423939'],"{'retweet_count': 2, 'reply_count': 0, 'like_c...",CVE-2022-42889
4,2023-02-06T15:02:32.000Z,1622611709390389248,CVE-2017-2997 https://t.co/lC3cUVrBN9 #HarsiaInfo,1.083501e+18,['1622611709390389248'],"{'retweet_count': 0, 'reply_count': 0, 'like_c...",CVE-2017-2997


## 1.1. Get the CVE-IDs which were mentioned by more than 10 authors

In [2]:
df_CVEs = df[["author_id","CVE"]].groupby(["CVE"])["author_id"]\
                                          .nunique()\
                                          .reset_index(name = 'num_of_author')\
                                          .sort_values(['num_of_author'],ascending = False)
df_CVEs.head()

,CVE,num_of_author
9508,CVE-2023-23397,3202
6285,CVE-2022-39952,1160
9024,CVE-2023-21716,883
9039,CVE-2023-21768,647
9653,CVE-2023-23752,612


In [4]:
len(df)

95012

In [5]:
len(df_CVEs)

11044

In [6]:
df_CVEs_10p = df_CVEs[df_CVEs["num_of_author"] >= 10]
len(df_CVEs_10p)

1724

## 1.2. Make a list of CVEs mentioned by more than 10 authors 

In [7]:
list_CVEs_10p = df_CVEs_10p["CVE"].tolist()

## 1.3. Get the CVSS Severity and Score in the list

In [8]:
from time import sleep
import logging
import requests
import json

# https://www.true-fly.com/entry/2022/03/02/073000
# 処理時間を計測するため、loggerのフォーマット設定しています
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
format = '%(asctime)s [%(levelname)s] %(filename)s, lines %(lineno)d. %(message)s'
formatter = logging.Formatter(format, '%Y-%m-%d %H:%M:%S')
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

cvss_severities = []
cvss_scores = []

def get_cvss_level(cve_id):
    # Initialize
    cvssV3 = []

    # 入力したCVE番号に対応する脆弱性情報をNVDのAPIからJSON形式で取得して、cvssV3のbaseSeverity, baseScoreを返却する
    api = "https://services.nvd.nist.gov/rest/json/cves/2.0\?cveId="
    cve = cve_id
    uri = api + cve
    # print(uri)
    try:
        response = requests.get(uri)
    except Exception as e :
        raise e
    # print(response.text)
    json_data = json.loads(response.text)
    
    if json_data['vulnerabilities']:
        baseSeverity = json_data['vulnerabilities'][0]['cve']['metrics']['cvssMetricV31'][0]['cvssData']['baseSeverity']
        baseScore = json_data['vulnerabilities'][0]['cve']['metrics']['cvssMetricV31'][0]['cvssData']['baseScore']
        cvssV3 = [baseSeverity,baseScore]
    
    else:
        baseSeverity = None
        baseScore = None
        cvssV3 = [baseSeverity,baseScore]

    return(cvssV3)

for n, cve in enumerate(list_CVEs_10p[:]):
    # API取得間隔（秒）
    # https://nvd.nist.gov/developers/start-here
    # "The public rate limit (without an API key) is 5 requests in a rolling 30 second window"
    interval = 6
    # リトライ間隔（秒）
    retry_interval = 6
    # 試行回数
    tries = 5
    
    cvss_level = []
    cvss_severity = None
    cvss_score = None
    
    print(n)
    
    for i in range(0, tries):
        try:
            cvss_level = get_cvss_level(cve)
            cvss_severity = cvss_level[0]
            cvss_score = cvss_level[1]
            sleep(interval)
            break
            
        except Exception as e:
            if i + 1 == tries:
                logger.info("取得失敗{}:{}個目".format(e,n))
            
            sleep(retry_interval)
            logger.info('リトライ回数:{}回目'.format(i + 1))
            continue
            
    if cvss_level:
        cvss_severities.append(cvss_severity)
        cvss_scores.append(cvss_score)
    else:
        cvss_severities.append(None)
        cvss_scores.append(None)
        

#print(cvss_severities)
#print(cvss_scores)

df_CVEs10p_cvss = df_CVEs_10p
df_CVEs10p_cvss["baseSeverity"] = cvss_severities
df_CVEs10p_cvss["baseScore"] = cvss_scores

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83


2023-04-06 04:41:30 [INFO] 2734311470.py, lines 76. リトライ回数:1回目
2023-04-06 04:41:37 [INFO] 2734311470.py, lines 76. リトライ回数:2回目
2023-04-06 04:41:44 [INFO] 2734311470.py, lines 76. リトライ回数:3回目
2023-04-06 04:41:52 [INFO] 2734311470.py, lines 76. リトライ回数:4回目
2023-04-06 04:41:53 [INFO] 2734311470.py, lines 73. 取得失敗'cvssMetricV31':83個目
2023-04-06 04:41:59 [INFO] 2734311470.py, lines 76. リトライ回数:5回目


84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119


2023-04-06 04:46:15 [INFO] 2734311470.py, lines 76. リトライ回数:1回目
2023-04-06 04:46:23 [INFO] 2734311470.py, lines 76. リトライ回数:2回目
2023-04-06 04:46:30 [INFO] 2734311470.py, lines 76. リトライ回数:3回目
2023-04-06 04:46:37 [INFO] 2734311470.py, lines 76. リトライ回数:4回目
2023-04-06 04:46:38 [INFO] 2734311470.py, lines 73. 取得失敗'cvssMetricV31':119個目
2023-04-06 04:46:44 [INFO] 2734311470.py, lines 76. リトライ回数:5回目


120
121
122
123
124
125
126
127
128
129
130
131


...

2023-04-06 08:00:47 [INFO] 2734311470.py, lines 76. リトライ回数:1回目
2023-04-06 08:00:54 [INFO] 2734311470.py, lines 76. リトライ回数:2回目
2023-04-06 08:01:08 [INFO] 2734311470.py, lines 76. リトライ回数:3回目
2023-04-06 08:01:15 [INFO] 2734311470.py, lines 76. リトライ回数:4回目
2023-04-06 08:01:18 [INFO] 2734311470.py, lines 73. 取得失敗'cvssMetricV31':1606個目
2023-04-06 08:01:24 [INFO] 2734311470.py, lines 76. リトライ回数:5回目


1607
1608
1609
1610
1611
1612
1613
1614
1615
1616
1617
1618
1619
1620
1621
1622
1623
1624
1625
1626
1627
1628
1629
1630
1631
1632
1633
1634
1635
1636
1637
1638


2023-04-06 08:05:22 [INFO] 2734311470.py, lines 76. リトライ回数:1回目
2023-04-06 08:05:29 [INFO] 2734311470.py, lines 76. リトライ回数:2回目
2023-04-06 08:05:37 [INFO] 2734311470.py, lines 76. リトライ回数:3回目
2023-04-06 08:05:44 [INFO] 2734311470.py, lines 76. リトライ回数:4回目
2023-04-06 08:05:45 [INFO] 2734311470.py, lines 73. 取得失敗'cvssMetricV31':1638個目
2023-04-06 08:05:51 [INFO] 2734311470.py, lines 76. リトライ回数:5回目


1639
1640
1641
1642
1643
1644
1645
1646
1647
1648
1649
1650
1651
1652
1653
1654
1655
1656
1657
1658
1659
1660
1661
1662
1663
1664
1665
1666
1667
1668
1669
1670
1671
1672
1673
1674
1675
1676
1677
1678
1679
1680
1681
1682
1683
1684
1685
1686
1687
1688
1689
1690
1691
1692
1693
1694
1695
1696
1697
1698
1699
1700
1701
1702
1703
1704
1705
1706
1707
1708
1709
1710
1711
1712
1713
1714
1715
1716
1717
1718
1719
1720
1721
1722
1723


/tmp/ipykernel_7751/2734311470.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_CVEs10p_cvss["baseSeverity"] = cvss_severities
/tmp/ipykernel_7751/2734311470.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_CVEs10p_cvss["baseScore"] = cvss_scores


# 2. Determine whether the CVSS Score is not yet provided or CVE-ID is reserved

In [9]:
# https://qiita.com/usop4/items/70b7db1acbd54112c428
import urllib.request
from bs4 import BeautifulSoup
from time import sleep
import logging

import requests
import json

# https://www.true-fly.com/entry/2022/03/02/073000
# 処理時間を計測するため、loggerのフォーマット設定しています
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
format = '%(asctime)s [%(levelname)s] %(filename)s, lines %(lineno)d. %(message)s'
formatter = logging.Formatter(format, '%Y-%m-%d %H:%M:%S')
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

def get_cve_reserved(cve_id):
    # Initialize
    resultsPerPage = None

    # 入力したCVE番号に対応する脆弱性情報をNVDのAPIからJSON形式で取得して、resultsPerPageを返却する
    api = "https://cveawg.mitre.org/api/cve-id/"
    cve = cve_id
    uri = api + cve
    # print(uri)
    try:
        response = requests.get(uri)
    except Exception as e :
        raise e
    # print(response.text)
    json_data = json.loads(response.text)
    
    try:
        if json_data['state']:
            return [cve, json_data['state']]
    
    except :
        return [cve, json_data['error']]

def get_cve_description(cve_id):
# https://qiita.com/usop4/items/70b7db1acbd54112c428

    # リトライ間隔（秒）
    retry_interval = 6
    # 試行回数
    tries = 5
    
    for i in range(0, tries):
        try:
            instance = urllib.request.urlopen("https://nvd.nist.gov/vuln/detail/"+cve)
            break

        except Exception as e:
            if i + 1 == tries:
                raise e
            
            sleep(retry_interval)
            logger.info('リトライ回数:{}回目'.format(i + 1))
            continue
            
    soup = BeautifulSoup(instance, "html.parser")
    score = soup.select_one("[data-testid=vuln-cvss3-panel-score]")
    desc = soup.select_one("[data-testid=vuln-cvss3-nist-vector-na]")

    if hasattr(score, "text"):
        return [cve, score.text.strip()]
        # print(f"{cve}, {score.text.strip()}")
    
    elif hasattr(desc, "text"):
        return [cve, desc.text.replace('\r','').replace('\n',' ').replace('\t','')]
        # print(f"{cve}, {desc.text.strip()}")

## 2.1. Create a new DataFrame that doesn't include the Severity and Score 

In [10]:
df_NoneSeverityCVEs=df_CVEs10p_cvss[df_CVEs10p_cvss['baseSeverity'].isin([None])]

## 2.2. Write CSV files for both CVEs and CVEs without Severity 

In [11]:
df_NoneSeverityCVEs.to_csv("non_severity_cves_20230326_determined_at_20230406.csv", header=True,index=False)
df_CVEs10p_cvss.to_csv("CVEs_Over10authors_20230326.csv", header=True,index=False)

## 2.3. Create a list of CVEs to determine 

In [1]:
import pandas as pd
df_NoneSeverityCVEs = pd.read_csv("non_severity_cves_20230326_determined_at_20230406.csv")
df_NoneSeverityCVEs.head()

,CVE,num_of_author,baseSeverity,baseScore
0,CVE-2023-25610,278,NaN,NaN
1,CVE-2023-24749,96,NaN,NaN
2,CVE-2023-28760,76,NaN,NaN
3,CVE-2023-2561,66,NaN,NaN
4,CVE-2023-28303,55,NaN,NaN


In [12]:
nonSeverityCVEs_list = df_NoneSeverityCVEs['CVE'].to_list()

## 2.4. Determine whether the CVSS Score is not yet provided or CVE-ID is reserved

In [13]:
interval = 2
result = []
for cve in nonSeverityCVEs_list[:]:

    desc = get_cve_description(cve)

    
    if desc :
        result.append(desc)
    
    else :
        reserved = get_cve_reserved(cve)   
        sleep(interval)

        
        if reserved :
            result.append(reserved)
print(result)

[['CVE-2023-25610', 'RESERVED'], ['CVE-2023-24749', 'RESERVED'], ['CVE-2023-28760', 'RESERVED'], ['CVE-2023-2561', 'CVE_ID_NOT_FOUND'], ['CVE-2023-28303', 'RESERVED'], ['CVE-2017-9140', '6.1 MEDIUM'], ['CVE-2022-22655', 'RESERVED'], ['CVE-2022-23529', 'NVD score not yet provided.'], ['CVE-2023-0045', 'RESERVED'], ['CVE-2023-24486', 'RESERVED'], ['CVE-2016-10190', '9.8 CRITICAL'], ['CVE-2014-12509', 'CVE_ID_NOT_FOUND'], ['CVE-2018-5093', '7.5 HIGH'], ['CVE-2023-0291', 'RESERVED'], ['CVE-2014-12508', 'CVE_ID_NOT_FOUND'], ['CVE-2015-2291', '7.8 HIGH'], ['CVE-2023-26463', 'RESERVED'], ['CVE-2017-0145', '8.1 HIGH'], ['CVE-2023-2575', 'CVE_ID_NOT_FOUND'], ['CVE-2021-33237', 'NVD score not yet provided.'], ['CVE-2022-43634', 'NVD score not yet provided.'], ['CVE-2023-27326', 'RESERVED'], ['CVE-2023-27537', 'NVD score not yet provided.'], ['CVE-2022-38217', 'RESERVED'], ['CVE-2014-8361', 'NVD score not yet provided.'], ['CVE-2023-1247', 'NVD score not yet provided.'], ['CVE-2021-3099', 'NVD sc

In [14]:
df_checked = pd.DataFrame(result, columns = ['CVE_checked', 'reason'])

In [17]:
df_checked

,CVE_checked,reason
0,CVE-2023-25610,RESERVED
1,CVE-2023-24749,RESERVED
2,CVE-2023-28760,RESERVED
3,CVE-2023-2561,CVE_ID_NOT_FOUND
4,CVE-2023-28303,RESERVED
5,CVE-2017-9140,6.1 MEDIUM
6,CVE-2022-22655,RESERVED
7,CVE-2022-23529,NVD score not yet provided.
8,CVE-2023-0045,RESERVED
9,CVE-2023-24486,RESERVED


In [39]:
df_checked.to_csv("checked_reasons_for_non_severity_cves_at_20230406.csv", header = True)

In [10]:
df_checked = pd.read_csv("checked_reasons_for_non_severity_cves_at_20230406.csv", usecols = ['CVE_checked','reason'])
df_checked

,CVE_checked,reason
0,CVE-2023-25610,RESERVED
1,CVE-2023-24749,RESERVED
2,CVE-2023-28760,RESERVED
3,CVE-2023-2561,CVE_ID_NOT_FOUND
4,CVE-2023-28303,RESERVED
5,CVE-2017-9140,6.1 MEDIUM
6,CVE-2022-22655,RESERVED
7,CVE-2022-23529,NVD score not yet provided.
8,CVE-2023-0045,RESERVED
9,CVE-2023-24486,RESERVED


In [15]:
df_gotScore = df_checked[df_checked['reason'] != (('CVE_ID_NOT_FOUND') or ('NVD score not yet provided.') or ('RESERVED'))]
df_gotScore

,CVE_checked,reason
0,CVE-2023-25610,RESERVED
1,CVE-2023-24749,RESERVED
2,CVE-2023-28760,RESERVED
4,CVE-2023-28303,RESERVED
5,CVE-2017-9140,6.1 MEDIUM
6,CVE-2022-22655,RESERVED
7,CVE-2022-23529,NVD score not yet provided.
8,CVE-2023-0045,RESERVED
9,CVE-2023-24486,RESERVED
10,CVE-2016-10190,9.8 CRITICAL


In [35]:
df_NoneSeverityCVEs.columns

Index(['level_0', 'index', 'CVE', 'num_of_author', 'baseSeverity',
       'baseScore'],
      dtype='object')

In [38]:
df_NoneSeverityCVEs = df_NoneSeverityCVEs[df_NoneSeverityCVEs['CVE', 'num_of_author', 'baseSeverity','baseScore']]


KeyError: ('CVE', 'num_of_author', 'baseSeverity', 'baseScore')

In [33]:
df_NoneSeverityCVEs.head()

,level_0,index,CVE,num_of_author,baseSeverity,baseScore
0,0,10267,CVE-2023-25610,278,None,NaN
1,1,10013,CVE-2023-24749,96,None,NaN
2,2,11030,CVE-2023-28760,76,None,NaN
3,3,10266,CVE-2023-2561,66,None,NaN
4,4,10927,CVE-2023-28303,55,None,NaN


In [28]:
df_checked.head()

,CVE_checked,reason
0,CVE-2023-25610,RESERVED
1,CVE-2023-24749,RESERVED
2,CVE-2023-28760,RESERVED
3,CVE-2023-2561,CVE_ID_NOT_FOUND
4,CVE-2023-28303,RESERVED


In [30]:
df_concat = pd.concat([df_NoneSeverityCVEs, df_checked],axis = 0)
df_concat.head

<bound method NDFrame.head of       index             CVE  num_of_author baseSeverity  baseScore  \
0   10267.0  CVE-2023-25610          278.0         None        NaN   
1   10013.0  CVE-2023-24749           96.0         None        NaN   
2   11030.0  CVE-2023-28760           76.0         None        NaN   
3   10266.0   CVE-2023-2561           66.0         None        NaN   
4   10927.0  CVE-2023-28303           55.0         None        NaN   
..      ...             ...            ...          ...        ...   
49      NaN             NaN            NaN          NaN        NaN   
50      NaN             NaN            NaN          NaN        NaN   
51      NaN             NaN            NaN          NaN        NaN   
52      NaN             NaN            NaN          NaN        NaN   
53      NaN             NaN            NaN          NaN        NaN   

       CVE_checked                       reason  
0              NaN                          NaN  
1              NaN           

In [16]:
df_concat.to_csv("checked_non_severity_cves_20230406.csv", header = True)

,CVE,num_of_author,baseSeverity,baseScore
10267,CVE-2023-25610,278,None,NaN
10013,CVE-2023-24749,96,None,NaN
11030,CVE-2023-28760,76,None,NaN
10266,CVE-2023-2561,66,None,NaN
10927,CVE-2023-28303,55,None,NaN
1206,CVE-2017-9140,47,None,NaN
5201,CVE-2022-22655,42,None,NaN
5249,CVE-2022-23529,29,None,NaN
7635,CVE-2023-0045,29,None,NaN
9923,CVE-2023-24486,28,None,NaN
